# The TEGR1 Data Set


This is a streamlined data ingress for 1 round of QF. This is the expected dataset from an organization running QF like GitCoin or Giveth. 

This data is being used to implement a digital twin environment for analysis with the Token Engineering Commons. 

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
import param as pm

## Loading The Data

In [ ]:
df = pd.read_csv('output/TEGR1.csv')
df.head()

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,last_score_timestamp,rawScore,threshold,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,2023-04-25 13:48:59.888771+00:00,28.57,15,NaN,NaN,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,2023-04-25 16:55:55.447871+00:00,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,2023-04-25 17:25:19.667155+00:00,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN


Number of Donations, Number of Projects, and Number of Unique donors. All as integers. 

In [ ]:
# Inspect number of unique addresses
df.select_dtypes(include=['object']).nunique()

id                      253
projectId                16
roundId                   1
token                     2
voter                    83
grantAddress             16
last_score_timestamp     83
dtype: int64

Overview of Quadradic Funding Data

In [ ]:
df[['amountUSD', 'coefficient', 'rawScore']].describe()

,amountUSD,coefficient,rawScore
count,253.000000,253.000000,253.000000
mean,23.990991,1.239130,32.417589
std,83.592891,0.250259,11.695007
min,0.368890,1.000000,15.220000
25%,3.000196,1.000000,23.560000
50%,5.998856,1.000000,29.740000
75%,12.346883,1.500000,40.480000
max,936.545861,1.500000,55.060000


Overview of the TEC SME data.

In [ ]:
df[['balance_tec', 'tec_tokens_flag', 'balance_tea', 'tea_flag']].describe()

,balance_tec,tec_tokens_flag,balance_tea,tea_flag
count,108.000000,108.0,44.000000,44.0
mean,12899.797337,1.0,1.568182,1.0
std,20738.626327,0.0,2.224574,0.0
min,50.480437,1.0,-1.000000,1.0
25%,142.522074,1.0,-1.000000,1.0
50%,4483.885742,1.0,2.000000,1.0
75%,15565.761361,1.0,3.000000,1.0
max,73838.661487,1.0,5.000000,1.0


## Explore the Vote Coefficients Input Dataset Using Hvplot Explorer

In [ ]:
hvexplorer = hvplot.explorer(
    df, 
    height=400)
hvexplorer.param.set_param(kind='step', x='index', y_multi=['amountUSD', 'rawScore'], by=[])
hvexplorer.labels.title = 'TEC Quadratic Funding Round #1 Data'
hvexplorer.labels.xlabel = 'Index'
hvexplorer.labels.ylabel = 'USD Amount and Raw Score'
hvexplorer

hvDataFrameExplorer(axes=Axes(explorer=..., height=400, legend='right', logx=False, logy=False, name='Axes00821', responsive=False, shared_axes=True, width=None, xlim=None, ylim=None), by=[], colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping00881', rescale_discrete_levels=True, symmetric=False), groupby=[], kind='step', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels00928', rot=0, title='TEC Quadratic Funding Round #1 Data', xlabel='Index', ylabel='USD Amount and Raw Score'), name='hvDataFrameExplorer00788', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations00972', rasterize=False, x_sampling=None, y_sampling=None), style=Style(alpha=1, explorer=..., name='Style01016'), x='index', y='id', y_multi=['amountUSD', 'rawScore'], z=None)

In [ ]:
# hvexplorer.param.set_param(kind='scatter', x='rawScore', y_multi=['amountUSD'], by=['projectId'])
# hvexplorer.labels.xlabel = 'Raw Score'
# hvexplorer.labels.ylabel = 'Amount USD'

## Applying the QF Algorithm
Inspired by Octopus at https://forum.tecommons.org/t/strengths-and-weaknesses-of-conviction-voting-and-other-mechanisms/1278

```python
def quadratic_fund(str: project) -> float:
    allocation = np.square(np.sum([sqrt(agent.votes.get(project)) for agent in system.agents]))
    return allocation
```

In [ ]:
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,last_score_timestamp,rawScore,threshold,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,2023-04-25 13:48:59.888771+00:00,28.57,15,NaN,NaN,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,2023-04-25 16:55:55.447871+00:00,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,2023-04-25 17:25:19.667155+00:00,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,2023-05-09 22:47:57.682680+00:00,29.74,15,NaN,NaN,NaN,NaN
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,2023-05-09 23:24:15.932770+00:00,18.04,15,NaN,NaN,NaN,NaN


In [ ]:
df['sqrt(amountUSD)'] = np.sqrt(df['amountUSD'])
df['sum(sqrt(amountUSD))'] = df.groupby('projectId')['sqrt(amountUSD)'].transform('sum')
df['sq(sum(sqrt(amountUSD)))'] = df['sum(sqrt(amountUSD))'].transform(lambda x: x**2)
df['quadradic_allocation'] = df['sq(sum(sqrt(amountUSD)))'] / df['sq(sum(sqrt(amountUSD)))'].sum()
df['default_allocation'] = df['amountUSD'] / df['amountUSD'].sum()
# df['quadratic_amount'] = df['amountUSD'].sum() * df['quadradic_allocation']

In [ ]:
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,threshold,balance_tec,tec_tokens_flag,balance_tea,tea_flag,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadradic_allocation,default_allocation
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,...,15,NaN,NaN,3.0,1.0,3.030566,60.250452,3630.117021,0.002959,0.001513
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,...,15,NaN,NaN,NaN,NaN,2.023504,15.075191,227.261396,0.000185,0.000675
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,...,15,NaN,NaN,NaN,NaN,8.628248,56.397388,3180.665361,0.002593,0.012265
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,...,15,NaN,NaN,NaN,NaN,2.362943,47.809680,2285.765459,0.001863,0.000920
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,...,15,NaN,NaN,NaN,NaN,2.362943,62.051660,3850.408505,0.003139,0.000920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,...,15,NaN,NaN,NaN,NaN,1.359339,56.397388,3180.665361,0.002593,0.000304
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,...,15,NaN,NaN,NaN,NaN,1.357863,47.809680,2285.765459,0.001863,0.000304
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,...,15,NaN,NaN,NaN,NaN,1.357863,42.547549,1810.293913,0.001476,0.000304
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,...,15,NaN,NaN,NaN,NaN,1.357863,7.801686,60.866307,0.000050,0.000304


## Automating the algorithm.

In [ ]:
df = pd.read_csv('output/TEGR1.csv')
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,last_score_timestamp,rawScore,threshold,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,2023-04-25 13:48:59.888771+00:00,28.57,15,NaN,NaN,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,2023-04-25 16:55:55.447871+00:00,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,2023-04-25 17:25:19.667155+00:00,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,2023-05-09 22:47:57.682680+00:00,29.74,15,NaN,NaN,NaN,NaN
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,2023-05-09 23:24:15.932770+00:00,18.04,15,NaN,NaN,NaN,NaN


### The QF transformer.

In [ ]:
def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
    df = df.copy(deep=True)
    df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
    df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
    df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
    df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
    df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
    df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
    
    return df

In [ ]:
df = qf(df)
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,balance_tec,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadratic_amount_allocation,quadratic_amount
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,...,NaN,NaN,3.0,1.0,0.001513,3.030566,60.250452,3630.117021,0.002959,17.961853
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,...,NaN,NaN,NaN,NaN,0.000675,2.023504,15.075191,227.261396,0.000185,1.124492
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,...,NaN,NaN,NaN,NaN,0.012265,8.628248,56.397388,3180.665361,0.002593,15.737962
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,...,NaN,NaN,NaN,NaN,0.000920,2.362943,47.809680,2285.765459,0.001863,11.309989
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,...,NaN,NaN,NaN,NaN,0.000920,2.362943,62.051660,3850.408505,0.003139,19.051858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,...,NaN,NaN,NaN,NaN,0.000304,1.359339,56.397388,3180.665361,0.002593,15.737962
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,...,NaN,NaN,NaN,NaN,0.000304,1.357863,47.809680,2285.765459,0.001863,11.309989
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,...,NaN,NaN,NaN,NaN,0.000304,1.357863,42.547549,1810.293913,0.001476,8.957351
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,...,NaN,NaN,NaN,NaN,0.000304,1.357863,7.801686,60.866307,0.000050,0.301167


### SME Sginal Boosting

In [ ]:
# Compute the boosted allocation
df['amount_boosted'] = df['amountUSD'] * df['coefficient']
df = qf(df, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,sq(sum(sqrt(amountUSD))),quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted))),quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,...,3630.117021,0.002959,17.961853,13.776499,0.001796,3.711671,70.440067,4961.803089,0.003273,25.100027
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,...,227.261396,0.000185,1.124492,4.094567,0.000534,2.023504,16.321380,266.387447,0.000176,1.347561
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,...,3180.665361,0.002593,15.737962,74.446665,0.009707,8.628248,63.587712,4043.397162,0.002667,20.454132
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,...,2285.765459,0.001863,11.309989,5.583500,0.000728,2.362943,53.638466,2877.085023,0.001898,14.554167
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,...,3850.408505,0.003139,19.051858,5.583500,0.000728,2.362943,70.476216,4966.897020,0.003276,25.125795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,...,3180.665361,0.002593,15.737962,1.847803,0.000241,1.359339,63.587712,4043.397162,0.002667,20.454132
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,...,2285.765459,0.001863,11.309989,1.843793,0.000240,1.357863,53.638466,2877.085023,0.001898,14.554167
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,...,1810.293913,0.001476,8.957351,1.843793,0.000240,1.357863,45.042187,2028.798632,0.001338,10.262983
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,...,60.866307,0.000050,0.301167,1.843793,0.000240,1.357863,8.337738,69.517878,0.000046,0.351667


### The Allocations Dataset
Sum qf allocations by project.

In [ ]:
# Examine the project allocations
allocations = df[['projectId'] + list(df.columns[df.columns.str.contains('allocation')])].groupby('projectId').sum().drop('amount_boosted_allocation',axis=1)
allocations

,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount_boosted_allocation
projectId,,,
0x10b3f00e,0.007163,0.001570,0.001574
0x23387567,0.165409,0.037399,0.043275
0x4cd41869,0.083889,0.177688,0.147632
0x5351510d,0.008092,0.002346,0.002407
0x64a30a4b,0.059451,0.050307,0.055636
0x72b0d6a6,0.011525,0.004762,0.005334
0x8d6f0c7b,0.037947,0.083346,0.090515
0x97589cd1,0.045227,0.049264,0.050672
0xa9bdf738,0.031684,0.078471,0.081902


Verify that all allocations sum to one.

In [ ]:
allocations.sum()

amountUSD_allocation                   1.0
quadratic_amount_allocation            1.0
quadratic_amount_boosted_allocation    1.0
dtype: float64

### The Results Dataset

In [ ]:
donations = df['amountUSD'].sum()
df_donations = allocations * donations

matching_pool = 25_000
funding_pool = matching_pool + donations
df_funding_pool = allocations * funding_pool

results = df_donations.merge(df_funding_pool, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
results = results.sort_values('quadratic_amount_allocation_unmatched', ascending=False)

In [ ]:
results

,amountUSD_allocation_unmatched,quadratic_amount_allocation_unmatched,quadratic_amount_boosted_allocation_unmatched,amountUSD_allocation_matched,quadratic_amount_allocation_matched,quadratic_amount_boosted_allocation_matched
projectId,,,,,,
0xe6424ab2,611.890594,1379.771324,1466.984320,3132.149048,7062.781288,7509.207669
0x4cd41869,509.183286,1078.514627,896.082796,2606.410294,5520.706798,4586.873706
0xdd9b885d,2021.307778,1013.750531,939.944684,10346.681735,5189.191974,4811.394188
0x8d6f0c7b,230.329281,505.888334,549.398168,1179.010830,2589.544079,2812.262460
0xa9bdf738,192.313869,476.296440,497.121033,984.417323,2438.068922,2544.665969
0xe8249a10,263.524009,413.356603,445.464263,1348.928192,2115.892128,2280.245001
0x64a30a4b,360.852626,305.351505,337.695615,1847.134471,1563.035020,1728.598230
0x97589cd1,274.514986,299.021282,307.565046,1405.188871,1530.631836,1574.365703
0x23387567,1003.988620,227.002239,262.668946,5139.222650,1161.980352,1344.551294


## Visualizing the Results

Replicating Bar Chart #1 from the Medium Article.

In [ ]:
results.rename({
    'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
    'quadratic_amount_allocation_matched': 'QF Matched', 
    'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
}, axis=1).hvplot.bar(
    y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
    rot=45,
    stacked=False,
    title="Adding Expertise into the QF Signal",
).opts(multi_level=False)

:Bars   [projectId,Variable]   (value)

Replicating Bar Chart #2 from the Medium Article.

In [ ]:
results['Percentage Boost'] = 100 * ((results['quadratic_amount_boosted_allocation_matched'] - results['quadratic_amount_allocation_matched']) / results['quadratic_amount_allocation_matched'] + 1)

In [ ]:

results.hvplot.bar(
    y='Percentage Boost', 
    color='purple', 
    ylim=(0, 180), 
    yformatter="%.0f%%", 
    yticks=list(range(0,200,20)),
    grid=True,
    height=400,
    rot=45,
    title="SME Boost as % of QF Boost by Project",
) * hv.HLine(100)

:Overlay
   .Bars.I  :Bars   [projectId]   (Percentage Boost)
   .HLine.I :HLine   [x,y]

In [ ]:
results

,amountUSD_allocation_unmatched,quadratic_amount_allocation_unmatched,quadratic_amount_boosted_allocation_unmatched,amountUSD_allocation_matched,quadratic_amount_allocation_matched,quadratic_amount_boosted_allocation_matched,Percentage Boost
projectId,,,,,,,
0xe6424ab2,611.890594,1379.771324,1466.984320,3132.149048,7062.781288,7509.207669,106.320830
0x4cd41869,509.183286,1078.514627,896.082796,2606.410294,5520.706798,4586.873706,83.084900
0xdd9b885d,2021.307778,1013.750531,939.944684,10346.681735,5189.191974,4811.394188,92.719526
0x8d6f0c7b,230.329281,505.888334,549.398168,1179.010830,2589.544079,2812.262460,108.600679
0xa9bdf738,192.313869,476.296440,497.121033,984.417323,2438.068922,2544.665969,104.372192
0xe8249a10,263.524009,413.356603,445.464263,1348.928192,2115.892128,2280.245001,107.767545
0x64a30a4b,360.852626,305.351505,337.695615,1847.134471,1563.035020,1728.598230,110.592418
0x97589cd1,274.514986,299.021282,307.565046,1405.188871,1530.631836,1574.365703,102.857243
0x23387567,1003.988620,227.002239,262.668946,5139.222650,1161.980352,1344.551294,115.712051


# Parameterization and App Development

In [ ]:
class TECQFSME(pm.Parameterized):
    boosting = pm.Number(1, bounds=(0,2), step=0.1)
    dataset  = pm.DataFrame(precedence=-1)
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000)
    donations = pm.Number(0, constant=True)
    funding_pool = pm.Number(0, constant=True)
    allocations  = pm.DataFrame(precedence=-1)
    results  = pm.DataFrame(precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)  
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @pm.depends('boosting', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.donations = self.dataset['amountUSD'].sum()
            self.funding_pool = self.matching_pool + self.donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Update the Coefficient
            self.dataset['coefficient'] = 1 + self.boosting * (self.dataset['tec_tokens_flag'].astype(int) | self.dataset['tea_flag'].astype(int))

            # Apply the Coefficient
            self.dataset['amount_boosted'] = self.dataset['amountUSD'] * self.dataset['coefficient']

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Examine the project allocations
            self.allocations = self.dataset[['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])].groupby('projectId').sum().drop('amount_boosted_allocation',axis=1)

            # Save the results sort by quadratic funding amounts
            self.results = (self.donations * self.allocations).merge((self.funding_pool * self.allocations), left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            self.results = self.results.sort_values('quadratic_amount_allocation_unmatched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)

        
    @pm.depends('dataset')
    def view_expertise_signal(self):
        return self.results.rename({
            'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
            'quadratic_amount_allocation_matched': 'QF Matched', 
            'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
        }, axis=1).hvplot.bar(
            y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
            rot=45,
            stacked=False,
            title="Adding Expertise into the QF Signal",
        ).opts(multi_level=False, legend_position='top_right')
    
    @pm.depends('dataset')
    def view_percentage_boost(self):
        return self.results.hvplot.bar(
            y='Percentage Boost', 
            color='purple', 
            ylim=(0, 180), 
            yformatter="%.0f%%", 
            yticks=list(range(0,200,20)),
            grid=True,
            height=400,
            rot=45,
            title="SME Boost as % of QF Boost by Project",
        ) * hv.HLine(100)
        
    
    def view(self):
        return pn.Row(self.param, self.view_percentage_boost)

In [ ]:
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,sq(sum(sqrt(amountUSD))),quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted))),quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,...,3630.117021,0.002959,17.961853,13.776499,0.001796,3.711671,70.440067,4961.803089,0.003273,25.100027
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,...,227.261396,0.000185,1.124492,4.094567,0.000534,2.023504,16.321380,266.387447,0.000176,1.347561
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,...,3180.665361,0.002593,15.737962,74.446665,0.009707,8.628248,63.587712,4043.397162,0.002667,20.454132
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,...,2285.765459,0.001863,11.309989,5.583500,0.000728,2.362943,53.638466,2877.085023,0.001898,14.554167
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,...,3850.408505,0.003139,19.051858,5.583500,0.000728,2.362943,70.476216,4966.897020,0.003276,25.125795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,...,3180.665361,0.002593,15.737962,1.847803,0.000241,1.359339,63.587712,4043.397162,0.002667,20.454132
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,...,2285.765459,0.001863,11.309989,1.843793,0.000240,1.357863,53.638466,2877.085023,0.001898,14.554167
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,...,1810.293913,0.001476,8.957351,1.843793,0.000240,1.357863,45.042187,2028.798632,0.001338,10.262983
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,...,60.866307,0.000050,0.301167,1.843793,0.000240,1.357863,8.337738,69.517878,0.000046,0.351667


In [ ]:
tec_qf_sme = TECQFSME(dataset=df.replace(np.nan, 0).copy(deep=True))

In [ ]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=2, name='Boosting', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(disabled=True, name='Donations', value=6069.72067821)
        [4] FloatInput(disabled=True, name='Funding pool', value=31069.72067821)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [ ]:
tec_qf_sme.view_expertise_signal()

:Bars   [projectId,Variable]   (value)